In [214]:
import pandas as pd
from dateutil.parser import parse
from datetime import datetime
import datetime as dt
import math
# import numpy library
import numpy as np

df = pd.read_excel("FIS FINAL ARRIVALS WEAPONS SHEET 01AUG23.xlsx", header=2).iloc[:, :8]
endrow = list(df['GATE*']).index("TOTAL=")
df = df.iloc[:endrow]

df = df.drop(df.loc[df["FIS PAX*"] == "PRECLEAR"].index) # type: ignore
df = df[df["IAB PAX*"] > 0]

df["ETA*"] = df["ETA*"].apply(lambda x: str(x)[:4])
df['ETA*'] = df["ETA*"].apply(lambda x: datetime.strptime(x, "%H%M"))
df['deplane time'] = (df['IAB PAX*']+df["FIS PAX*"])/17
df["fisratio"] = df['FIS PAX*']/(df['IAB PAX*']+df["FIS PAX*"])

flt_ratio = dict(zip(df['FLIGHT*'], df['fisratio']))

In [215]:
df

,FLIGHT*,FROM,ETA*,GATE*,IAB PAX*,IAB BAGS,FIS PAX*,FIS BAGS*,deplane time,fisratio
0,261,MAD,1900-01-01 13:34:00,C1,80,77,144,150,13.176471,0.642857
1,123,LHR,1900-01-01 13:45:00,C8,81,87,81,89,9.529412,0.5
2,991,BCN,1900-01-01 14:05:00,C5,106,115,169,181,16.176471,0.614545
3,951,BRU,1900-01-01 14:29:00,C4,125,172,220,294,20.294118,0.637681
4,885,FCO,1900-01-01 14:24:00,C12,135,136,140,121,16.176471,0.509091
5,947,AMS,1900-01-01 14:35:00,C6,120,110,104,84,13.176471,0.464286
6,1567,MEX,1900-01-01 14:30:00,C9,117,129,30,26,8.647059,0.204082
7,975,GVA,1900-01-01 14:41:00,C3,67,66,94,83,9.470588,0.583851
8,914,CDG,1900-01-01 14:54:00,C2,129,150,146,130,16.176471,0.530909
9,943,LIS,1900-01-01 15:12:00,C8,63,52,163,140,13.294118,0.721239


In [216]:
planecap = dict(zip(df['FLIGHT*'], df['FIS PAX*'].add(df["IAB PAX*"])))

In [217]:
fltnum_deplane_time = dict(zip(df["FLIGHT*"], df["deplane time"]))
fltnum_eta = dict(zip(df["FLIGHT*"], df["ETA*"]))

In [218]:
timelist = pd.date_range(start='1900-01-01 13:00', end='1900-01-01 21:00', freq='1min').to_list()
indices = list(range(len(timelist)))
dt_to_inx = dict(zip(timelist, indices))

In [219]:
len(timelist)

481

In [220]:
def get_start_time_inx(fltnum):
    ts = fltnum_eta[fltnum]
    return dt_to_inx[ts]

In [221]:
DEPLANEMENT_PER_MIN = 20

In [222]:
def get_list_deplanement(flt):
    deplane_len = fltnum_deplane_time[flt]
    cap = planecap[flt]
    tlist = [DEPLANEMENT_PER_MIN for i in range(math.floor(deplane_len)-1)]
    remainder = cap - sum(tlist)
    tlist.append(remainder)
    return tlist
    



In [223]:


# define a function that pads a list with zeros
def pad_list(lst, k, n):
  # convert the list to a numpy array
  arr = np.array(lst)
  # calculate the number of zeros to add before and after the array
  before = k
  after = n - k - len(arr)
  # check if the padding is valid
  if before < 0 or after < 0:
    return "Invalid padding"
  # pad the array with zeros using np.pad()
  padded_arr = np.pad(arr, (before, after), mode="constant", constant_values=0)
  # convert the padded array back to a list
  padded_lst = padded_arr.tolist()
  # return the padded list
  return padded_lst



In [224]:
def get_lists(flights):
    deplanement_matrix = []
    for flightnum in flights:
        flt_deplanements = get_list_deplanement(flightnum)
        
        
        fisratio = flt_ratio[flightnum]
        
        
        
        padded_list = pad_list(flt_deplanements, get_start_time_inx(flightnum),len(timelist))

        nppad = np.array(padded_list)*fisratio
        
        padded_list = list(np.ceil(nppad))
        deplanement_matrix.append(padded_list)
    return deplanement_matrix

In [225]:
lists = get_lists(df['FLIGHT*'])

In [226]:
ndf = pd.DataFrame(pd.date_range(start='1900-01-01 13:00', end='1900-01-01 21:00', freq='1min'))
pmin = list(pd.DataFrame(lists).sum())

In [227]:
ndf = ndf.set_index(0)
ndf["pax/min"] = pmin

In [228]:
sum(ndf['pax/min'])

2491.0

In [229]:
sum(df['FIS PAX*'])

2366

In [230]:
ndf

,pax/min
0,
1900-01-01 13:00:00,0.0
1900-01-01 13:01:00,0.0
1900-01-01 13:02:00,0.0
1900-01-01 13:03:00,0.0
1900-01-01 13:04:00,0.0
...,...
1900-01-01 20:56:00,0.0
1900-01-01 20:57:00,0.0
1900-01-01 20:58:00,0.0


,pax/min
0,
1900-01-01 13:45:00,23.0
1900-01-01 14:29:00,24.0
1900-01-01 14:30:00,29.0
1900-01-01 14:31:00,29.0
1900-01-01 14:32:00,29.0
...,...
1900-01-01 16:00:00,21.0
1900-01-01 16:01:00,21.0
1900-01-01 16:02:00,21.0


In [234]:
df.to_csv("edited.csv")